<a href="https://colab.research.google.com/github/theo-chengkai-wang/AISF_project/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration Phase

This part documents the exploration phase based on Neel Nanda's exploration

## Setup

### Environment Setup (ignore)

**You can ignore this part:** It's just for use internally to setup the tutorial in different
environments. You can delete this section if using in your own repo.

In [1]:

# # Detect if we're running in Google Colab
# try:
#     import google.colab
#     IN_COLAB = True
#     print("Running as a Colab notebook")
# except:
#     IN_COLAB = False

# # Install if in Colab
# if IN_COLAB:
#     %pip install transformer_lens
#     %pip install circuitsvis
#     # Install a faster Node version
#     !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# # Hot reload in development mode & not running on the CD
# if not IN_COLAB:
#     from IPython import get_ipython
#     ip = get_ipython()
#     if not ip.extension_manager.loaded:
#         ip.extension_manager.load('autoreload')
#         %autoreload 2


%pip install transformer_lens
%pip install circuitsvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    

### Imports

In [2]:
from functools import partial
from typing import List, Optional, Union

import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import torch
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float

import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

In [3]:
import string
import torch.nn.functional as F
import pandas as pd

### PyTorch Setup

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [4]:
torch.set_grad_enabled(False)
print("Disabled automatic differentiation")

Disabled automatic differentiation


### Plotting Helper Functions (ignore)

Some plotting helper functions are included here (for simplicity).

In [5]:
def imshow(tensor, **kwargs):
    px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        **kwargs,
    ).show()


def line(tensor, **kwargs):
    px.line(
        y=utils.to_numpy(tensor),
        **kwargs,
    ).show()


def scatter(x, y, xaxis="", yaxis="", caxis="", **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(
        y=y,
        x=x,
        labels={"x": xaxis, "y": yaxis, "color": caxis},
        **kwargs,
    ).show()

## First Exploration

### Setup

In [6]:
# NBVAL_IGNORE_OUTPUT
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

# Get the default device used
device: torch.device = utils.get_device()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


### Playing with prompts

In [ ]:
# example_prompt = "Friends Union Party (FUP), Zoo Vermont Emirates ("
# example_answer = ""
# 2
example_prompt = "Friends Union Party (FUP), Arizona State University ("
example_answer = "AS"
# 3
p =     "Mary is a member of the Question Answer Team (QAT), as well as the Motor Assistance Group ("
a = "V"
# 1
utils.test_prompt(p, a, model, prepend_space_to_answer=False, prepend_bos=True)

In [ ]:
prompts = [
    "Mary is a member of the Friends Union Party (FUP), as well as the Carry On Girls (",
    "John is a member of the Coal Mining Institute (CMI), as well as the Area Effect University (",
    "John is a member of the Horror Film Society (HFS), as well as the Bar Beer Parish (",
    "Mary is a member of the Coal Mining Institute (CMI), as well as the National Executive Board (",
    "Aaron is a member of the Friends Union Party (FUP), as well as the Direct Access Corp (",
    "Kate is a member of the Genetic Mod Plant (GMP), as well as the Electronic Killer Organisation (",
    "Kate is a member of the Coal Mining Institute (CMI), as well as the Zero Waste Initiative (",
    "Bill is a member of the Zero Waste Initiative (ZWI), as well as the Genetic Mod Plant (",
    "Luke is a member of the Genetic Mod Plant (GMP), as well as the Political Onion Party (",
    "Billy is a member of the Political Onion Party (POP), as well as the South Oxford School (",
    "Harry is a member of the Genetic Mod Plant (GMP), as well as the Yellow Fall Board (",
    "Mary is a member of the Yellow Fall Board (YFB), as well as the Friends Union Party (",
    "Mary is a member of the Friends Union Party (FUP), as well as the Horror Film Society (",
    "Mary is a member of the Horror Film Society (HFS), as well as the X Ray Institute (",
    "Mary is a member of the X Ray Institute (XRI), as well as the World Trade Centre (",
    "Billy is a member of the Political Onion Party (POP), as well as the International Wii Board (",
    "Billy is a member of the International Wii Board (IWB), as well as the Reality Doom Arena (",
    "Billy is a member of the Reality Doom Arena (RDA), as well as the Killer Game Society (",
    "Mary is a member of the Killer Game Society (KGS), as well as the Question Answer Team (",
    "Mary is a member of the Question Answer Team (QAT), as well as the Motor Assistance Group (",
    "Billy is a member of the Question Answer Team (QAT), as well as the Technology Application Course (",
    "Mary is a member of the Technology Application Course (TAC), as well as the Local Football Club (",
    "Billy is a member of the Technology Application Course (TAC), as well as the Video Games Club (",
    "Billy is a member of the Video Games Club (VGC), as well as the Object Information Network (",
    "Billy is a member of the Video Games Club (VGC), as well as the Junior Common Room (",
    "Billy is a member of the Video Games Club (VGC), as well as the Uniform Distribution Fans ("
]

answers = [
    ["C", "F"],
    ["A", "C"],
    ["B", "A"],
    ["N", "B"],
    ["D", "N"],
    ["E", "D"],
    ["Z", "C"],
    ["G", "Z"],
    ["P", "G"],
    ["S", "P"],
    ["Y", "G"],
    ["F", "Y"],
    ["H", "F"],
    ["X", "H"],
    ["W", "X"],
    ["I", "P"],
    ["R", "I"],
    ["K", "R"],
    ["Q", "K"],
    ["M", "Q"],
    ["T", "Q"],
    ["L", "T"],
    ["V", "T"],
    ["O", "V"],
    ["J", "V"],
    ["U", "V"]
]

print([c for c in string.ascii_uppercase if c not in [l[0] for l in answers]])
print(len(prompts) == len(answers))

tokens = []
for prompt in prompts:
    str_tokens = model.to_str_tokens(prompt)
    print("Prompt length:", len(str_tokens))
    print("Prompt as tokens:", str_tokens)
    tokens.append(str_tokens)


In [ ]:
# Now setup everything!
# List of the token (ie an integer) corresponding to each answer, in the format (correct_token, incorrect_token)
answer_tokens = [[model.to_single_token(c) for c in a] for a in answers]
answer_tokens = torch.tensor(answer_tokens).to(device)

### Forming Hypotheses

We first make a few observations.

- Due to tokenization, have common capital letter combinations together as one token. But we want to study simple behaviour, so we'll only restrict our experiments to a restricted task of finding the first letter.
- Also there are cases where instead of giving one letter or two letters as the initials, the top choice is a canonical combination of letters. E.g. CIA, but also e.g. Arizona ⇒ AR, California ⇒ CA as the US state codes. Thus we want to avoid these proper nouns.
    - Moreover there are weird influences: HBO ⇒ Gives Netflix, Youtube etc; Nationwide ⇒ NYSE, banking related things etc.
- The model only performs well if the name looks like that of an 'organisation'. If not, it will not resort to the behaviour of generating an acronym.
- Another assumption we will make is that the baseline behaviour is copying: if the task was unclear or there wasn't sufficient information, often the model will copy the first token after the parenthesis before. This is our baseline for logit difference comparison (instead of working w the absolute value of logits.)

The precise task we look at is: given a 3-word proper name that looks like an *organisation*, output the first letter of the first word. The prompt would looks like

```Mary is a member of the Friends Union Party (FUP), as well as the Carry On Girls (```

where the "member" token should give information about the choice of task, and "the Friends Union Party (FUP)" is an example, making sure that the model knows which task it has to do. Moreover, we make sure that the example acronym "FUP" is tokenised as `["F", "UP"]`. This is important for uniformity and also to isolate out the notion of really giving the first letter individually.

Now let's try to form a hypothesis on how the model could work.

Firstly, just the task of predicting the first letter is probably doable by a skip trigram: `Carry ... ( -> C`. A more complicated question would then be HOW the model knows that it needs to look at `Carry`, and not `On`, `Girls`, or in fact directly perform copying `F` -- last time after `(` there was an `F`, so this time we also predict an `F`. In fact, predicting `F` is a common behaviour amongst the prompts I have tried.

For now, I lack an intuition of why this might be. Maybe there will be a head that moves all the capitalised words to `(`, and then the MLP will choose the right one to focus on; or maybe there is a head which inhibits all the words which are not the right one, i.e. the first word of the second three-word sequence in the sentence.

Let' s run experiments first.

## Exploratory Experiments based on Neel Nanda's experiments

In [ ]:
tokens = model.to_tokens(prompts, prepend_bos=True)

# Run the model and cache all activations
original_logits, cache = model.run_with_cache(tokens)

In [ ]:
original_logits.shape

In [12]:
def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
    # Only the final logits are relevant for the answer
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    if per_prompt:
        return answer_logit_diff
    else:
        return answer_logit_diff.mean()


print(
    "Per prompt logit difference:",
    logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True)
    .detach()
    .cpu()
    .round(decimals=3),
)
original_average_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
print(
    "Average logit difference:",
    round(logits_to_ave_logit_diff(original_logits, answer_tokens).item(), 3),
)

Per prompt logit difference: tensor([3.7390, 8.9690, 3.5600, 9.0990, 7.1310, 6.7220, 7.5080, 4.3920, 5.7880,
        5.0790, 3.7190, 3.0870, 2.3320, 2.9450, 7.4040, 5.0530, 4.9370, 2.9440,
        6.6510, 6.4630, 7.9690, 6.8520, 5.9820, 3.0690, 0.7770, 0.7490])
Average logit difference: 5.112


### Direct Logit Attribution

In [13]:
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)
# Unembedding vector of tokens, i.e. row in unembedding matrix corresponding to
# that token.
# print(answer_residual_directions)
print("Answer residual directions shape:", answer_residual_directions.shape)
logit_diff_directions = (
    answer_residual_directions[:, 0] - answer_residual_directions[:, 1]
)
print("Logit difference directions shape:", logit_diff_directions.shape)

Answer residual directions shape: torch.Size([26, 2, 768])
Logit difference directions shape: torch.Size([26, 768])


In [14]:
# cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type].
final_residual_stream = cache["resid_post", -1]
print("Final residual stream shape:", final_residual_stream.shape)
final_token_residual_stream = final_residual_stream[:, -1, :]
# Apply LayerNorm scaling
# pos_slice is the subset of the positions we take - here the final token of each prompt
scaled_final_token_residual_stream = cache.apply_ln_to_stack(
    final_token_residual_stream, layer=-1, pos_slice=-1
)
print(scaled_final_token_residual_stream.shape)

print(scaled_final_token_residual_stream)

print(logit_diff_directions)

print(einsum(
    "batch d_model, batch d_model -> batch ",
    scaled_final_token_residual_stream,
    logit_diff_directions,
))

average_logit_diff = einsum(
    "batch d_model, batch d_model -> ",
    scaled_final_token_residual_stream,
    logit_diff_directions,
) / len(prompts)
print("Calculated average logit diff:", round(average_logit_diff.item(), 3))
print("Original logit difference:", round(original_average_logit_diff.item(), 3))

Final residual stream shape: torch.Size([26, 22, 768])
torch.Size([26, 768])
tensor([[-0.1698,  0.2760,  0.4704,  ..., -0.1758,  0.2026, -0.1025],
        [-0.1940, -0.5171, -0.6501,  ...,  0.1382,  0.9492, -0.0063],
        [-0.2667, -0.0813,  0.3325,  ...,  0.0263,  0.1580, -0.3343],
        ...,
        [-0.3994, -0.3068, -0.3709,  ..., -0.1031,  0.3296,  0.5018],
        [ 0.0428, -0.3437,  0.6545,  ..., -0.1631,  0.4650, -0.3522],
        [-0.3553,  0.1704,  0.3380,  ..., -0.3137, -0.2611,  0.1856]],
       device='cuda:0')
tensor([[ 0.0717,  0.1734,  0.2904,  ...,  0.2114, -0.0200,  0.1677],
        [-0.3453, -0.2651, -0.3457,  ..., -0.0436,  0.0582,  0.0088],
        [ 0.3578,  0.0110,  0.2483,  ...,  0.0454, -0.1898, -0.0556],
        ...,
        [-0.1732,  0.2886, -0.2666,  ..., -0.2288,  0.2088,  0.0591],
        [ 0.1115,  0.1454, -0.3030,  ..., -0.3152,  0.1535, -0.1994],
        [-0.1920,  0.2327, -0.3971,  ..., -0.2003,  0.1998,  0.0182]],
       device='cuda:0')
tensor(

In [15]:
def residual_stack_to_logit_diff(
    residual_stack: Float[torch.Tensor, "components batch d_model"],
    cache: ActivationCache,
) -> float:
    scaled_residual_stack = cache.apply_ln_to_stack(
        residual_stack, layer=-1, pos_slice=-1
    )
    return einsum(
        "... batch d_model, batch d_model -> ...",
        scaled_residual_stack,
        logit_diff_directions,
    ) / len(prompts)

In [16]:
accumulated_residual, labels = cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True
)
print(accumulated_residual.shape, )
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, cache)
line(
    logit_lens_logit_diffs,
    x=np.arange(model.cfg.n_layers * 2 + 1) / 2,
    hover_name=labels,
    title="Logit Difference From Accumulate Residual Stream",
)

torch.Size([25, 26, 768])


What's really interesting so far is that the computation somehow starts from layer 8, and steadily increases (without spiking) up to layer 11/12. Now perhaps one should ask what exactly are the top $n$ tokens after each layer.

In [17]:
# print(accumulated_residual.shape)
def logit_lens_for_each_layer():
  acc_res, labels = cache.accumulated_resid(
      layer=-1, incl_mid=True, pos_slice=-1, return_labels=True, apply_ln=True
  )
  # scaled_residual_stack = cache.apply_ln_to_stack(
  #     acc_res, layer=-1, pos_slice=-1
  # )
  # print(scaled_residual_stack.shape) # layer, batch, d_model
  W_U = model.W_U
  layers_unembedded = einsum(
    "layer batch d_model, d_model d_vocab -> layer batch d_vocab",
    acc_res,
    W_U,
  ).cpu()
  logit_tops, top_idx = torch.topk(layers_unembedded, 3, dim=-1)
  return [[(labels[i], model.to_str_tokens(layer)) for i, layer in enumerate(top_idx[:, prompt, :])] for prompt in range(len(answers))]

In [18]:
# layers_unembedded = logit_lens_for_each_layer()
# logit_tops, top_idx = torch.topk(layers_unembedded, 3, dim=-1)

In [19]:
outcomes = logit_lens_for_each_layer()

Unfortunately the error due to the unorthodox application of layer norms caused the output tokens to become a COMBINATION of the correct first letter and another letter, instead of the actual answer. However, it's still quite telling that it's at the last few layers have 2-letter combinations that start with the right letter already.

In [20]:
outcomes

[[('0_pre', [' mathemat', ' corrid', ' challeng']),
  ('0_mid', ['ART', '19', 'talk']),
  ('1_pre', ['see', 'aka', 'sic']),
  ('1_mid', ['see', 'aka', '...)']),
  ('2_pre', ['...)', 'see', 'sic']),
  ('2_mid', ['...)', 'aka', '?)']),
  ('3_pre', ['sic', '...)', '?)']),
  ('3_mid', ['sic', 'aka', '…)']),
  ('4_pre', ['sic', 'aka', '...)']),
  ('4_mid', ['sic', 'aka', 'formerly']),
  ('5_pre', ['sic', 'aka', 'formerly']),
  ('5_mid', ['aka', 'sic', 'formerly']),
  ('6_pre', ['formerly', 'aka', 'sic']),
  ('6_mid', ['sic', 'aka', 'formerly']),
  ('7_pre', ['sic', 'aka', 'formerly']),
  ('7_mid', ['sic', 'aka', 'formerly']),
  ('8_pre', ['sic', 'aka', 'formerly']),
  ('8_mid', ['sic', 'aka', 'CW']),
  ('9_pre', ['CN', 'CW', 'C']),
  ('9_mid', ['CW', 'sic', 'CN']),
  ('10_pre', ['CW', 'CBC', 'CB']),
  ('10_mid', ['CB', 'CW', 'CC']),
  ('11_pre', ['CC', 'CB', 'CAR']),
  ('11_mid', ['CC', 'CF', 'CW']),
  ('final_post', ['CF', 'CC', 'CB'])],
 [('0_pre', [' mathemat', ' corrid', ' challeng']),


In [21]:
per_layer_residual, labels = cache.decompose_resid(
    layer=-1, pos_slice=-1, return_labels=True
)
print(per_layer_residual.shape, labels)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, cache)
line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

torch.Size([26, 26, 768]) ['embed', 'pos_embed', '0_attn_out', '0_mlp_out', '1_attn_out', '1_mlp_out', '2_attn_out', '2_mlp_out', '3_attn_out', '3_mlp_out', '4_attn_out', '4_mlp_out', '5_attn_out', '5_mlp_out', '6_attn_out', '6_mlp_out', '7_attn_out', '7_mlp_out', '8_attn_out', '8_mlp_out', '9_attn_out', '9_mlp_out', '10_attn_out', '10_mlp_out', '11_attn_out', '11_mlp_out']


If we look at the per-layer difference, unsurprisingly we have both contributions attention layers earlier, and then MLP layers later. Something significant seems to happen on L8 attn, L10 attn, and on L11 attn somehow the performance gets worse. Also, L9-11 MLPs seem to play a non-negligible role. We hypothesise these activities are used to convert the word to initial.

Now we follow Neel Nanda to do per-head attribution.

The observation is that here only a few heads matter. In fact L9H9, L10H10 and L8H11 truely seem to be the only ones which matter a lot, whereas on layer 10 and 11 there are also a couple of heads which decrease our final logits. Unclear why.

In [22]:
per_head_residual, labels = cache.stack_head_results(
    layer=-1, pos_slice=-1, return_labels=True
)
print(per_head_residual.shape)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache)
print(per_head_logit_diffs.shape)
per_head_logit_diffs = einops.rearrange(
    per_head_logit_diffs,
    "(layer head_index) -> layer head_index",
    layer=model.cfg.n_layers,
    head_index=model.cfg.n_heads,
)
print(per_head_logit_diffs.shape)
imshow(
    per_head_logit_diffs,
    labels={"x": "Head", "y": "Layer"},
    title="Logit Difference From Each Head",
)

Tried to stack head results when they weren't cached. Computing head results now
torch.Size([144, 26, 768])
torch.Size([144])
torch.Size([12, 12])


### Attention Analysis

One thing we could try to do is to look at the attention patterns of the important heads.

In [23]:
def visualize_attention_patterns(
    heads: Union[List[int], int, Float[torch.Tensor, "heads"]],
    local_cache: ActivationCache,
    local_tokens: torch.Tensor,
    title: Optional[str] = "",
    max_width: Optional[int] = 700,
) -> str:
    # If a single head is given, convert to a list
    if isinstance(heads, int):
        heads = [heads]

    # Create the plotting data
    labels: List[str] = []
    patterns: List[Float[torch.Tensor, "dest_pos src_pos"]] = []

    # Assume we have a single batch item
    batch_index = 0

    for head in heads:
        # Set the label
        layer = head // model.cfg.n_heads
        head_index = head % model.cfg.n_heads
        labels.append(f"L{layer}H{head_index}")

        # Get the attention patterns for the head
        # Attention patterns have shape [batch, head_index, query_pos, key_pos]
        patterns.append(local_cache["attn", layer][batch_index, head_index])

    # Convert the tokens to strings (for the axis labels)
    str_tokens = model.to_str_tokens(local_tokens)

    # Combine the patterns into a single tensor
    patterns: Float[torch.Tensor, "head_index dest_pos src_pos"] = torch.stack(
        patterns, dim=0
    )

    # Circuitsvis Plot (note we get the code version so we can concatenate with the title)
    plot = attention_heads(
        attention=patterns, tokens=str_tokens, attention_head_names=labels
    ).show_code()

    # Display the title
    title_html = f"<h2>{title}</h2><br/>"

    # Return the visualisation as raw code
    return f"<div style='max-width: {str(max_width)}px;'>{title_html + plot}</div>"

In [24]:
top_k = 3

top_positive_logit_attr_heads = torch.topk(
    per_head_logit_diffs.flatten(), k=top_k
).indices
print(top_positive_logit_attr_heads)

positive_html = visualize_attention_patterns(
    top_positive_logit_attr_heads,
    cache,
    tokens[0],
    f"Top {top_k} Positive Logit Attribution Heads",
)

top_negative_logit_attr_heads = torch.topk(
    -per_head_logit_diffs.flatten(), k=top_k
).indices
print(top_negative_logit_attr_heads)

negative_html = visualize_attention_patterns(
    top_negative_logit_attr_heads,
    cache,
    tokens[0],
    title=f"Top {top_k} Negative Logit Attribution Heads",
)

HTML(positive_html + negative_html)

tensor([130, 107, 117], device='cuda:0')
tensor([136, 127, 138], device='cuda:0')


Visualisation:
- All three of the top contributing heads seem to be moving information from the capitalised words to the tokens corresponding to the acronym.
- L10H10 clearly moves information from Carry to the parenthesis, but also `Friends`, `F`, and `On Girls`. As expected. Similar to L8H11. If we look at the first acronym and how attention works there, we clearly see an opposite-staircase sort of attention pattern: all three words attend to `(`, and then only the second and third words attend to the first token of the acronym and so on. We don't study this in more detail but I hypothesise that if a token is fixed, then the way the remaining two are generated is by doing exactly this, and if the token chosen is a 2-letter token, then the third one gets inhibited somehow.
- L9H9 looks very different. The other two all have big attention from the first word, Carry, to `(`. Not the case here. An attention pattern clearly with a much bigger emphasis for the second and third word of the acronym than the first -- storing this info in advance, BUT ALSO predicting the correct combination of letters if that exists (predict CIA instead of CI or C).
- In the negative ones L11H4 and L11H6 it's unclear.
- L10H7 is interesting though: you also have this attention pattern of a positive head. Though it looks more like a head which does some sort of skip-induction: After the first word in the acronym it moves information about `Party` to `On Girls` and from `Union` to `On`. I'm guessing this is used to predict the next word at those points, but what it does do is raise the probability of getting a copying behaviour.

### Activation Patching setup

In [25]:
%pip install git+https://github.com/neelnanda-io/neel-plotly.git

  Cloning https://github.com/neelnanda-io/neel-plotly.git to /tmp/pip-req-build-j0uc4oke
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/neel-plotly.git /tmp/pip-req-build-j0uc4oke
  Resolved https://github.com/neelnanda-io/neel-plotly.git to commit 6dc24b26f8dec991908479d7445dae496b3430b7
  Preparing metadata (setup.py) ... done
  Created wheel for neel_plotly: filename=neel_plotly-0.0.0-py3-none-any.whl size=10188 sha256=b730814e121c7e830ce82636642e37311b500fd43c42efd1990ccb717628b75e
  Stored in directory: /tmp/pip-ephem-wheel-cache-cp_zru35/wheels/32/cf/25/0103b4be02266c40faf008ffa9565a2ba07d1c63118fccc390
Successfully built neel_plotly


In [26]:
import transformer_lens.patching as patching
import neel_plotly

### Activation Patching attempt 1: localising the effects



In [ ]:
# First define all the clean and corrupted tokens

clean_tokens = model.to_tokens(prompts)

# Corrupted tokens are replacing the token 18 "Carry" by token 7 "Friends"
corrupted_tokens = clean_tokens.clone()
corrupted_tokens[:, 18] = clean_tokens[:, 7]

In [ ]:
print("Clean string 0", model.to_string(clean_tokens[0]))
print("Corrupted string 0", model.to_string(corrupted_tokens[0]))

In [ ]:
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)

In [ ]:
clean_difference = logits_to_ave_logit_diff(clean_logits, answer_tokens).cpu()
corrupted_difference = logits_to_ave_logit_diff(corrupted_logits, answer_tokens).cpu()

In [ ]:
print(clean_difference, corrupted_difference)

In [ ]:
def metric(logits, answer_token_indices=answer_tokens):
  return (logits_to_ave_logit_diff(logits, answer_token_indices).cpu() - corrupted_difference) / (clean_difference - corrupted_difference)

print(metric(clean_logits, answer_tokens))
print(metric(corrupted_logits, answer_tokens))

In [ ]:
resid_pre_act_patch_results = patching.get_act_patch_resid_pre(model, corrupted_tokens, clean_cache, metric)

In [ ]:
neel_plotly.imshow(resid_pre_act_patch_results,
       yaxis="Layer",
       xaxis="Position",
       x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
       title="resid_pre Activation Patching")

In [ ]:
attn_out_patch_results = patching.get_act_patch_attn_out(model, corrupted_tokens, clean_cache, metric)

  0%|          | 0/264 [00:00<?, ?it/s]

In [ ]:
print(attn_out_patch_results.shape)

torch.Size([12, 22])


In [ ]:
neel_plotly.imshow(attn_out_patch_results,
       yaxis="Layer",
       xaxis="Position",
       x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
       title="Attention Activation Patching")

In [ ]:
mlp_out_patch_results = patching.get_act_patch_mlp_out(model, corrupted_tokens, clean_cache, metric)

  0%|          | 0/264 [00:00<?, ?it/s]

In [ ]:
print(mlp_out_patch_results.shape)

torch.Size([12, 22])


In [ ]:
neel_plotly.imshow(mlp_out_patch_results,
       yaxis="Layer",
       xaxis="Position",
       x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
       title="MLP Activation Patching")

There is a transfer of information from the `Carry` token to `(` at layer 7 -> layer 8 via attention; and that information is then processed by layers 8, 9, 10 MLPs to give the prediction of "C".

Let's look at this transfer of information more deeply.

In [ ]:
every_head_all_pos_act_patch_result = patching.get_act_patch_attn_head_all_pos_every(model, corrupted_tokens, clean_cache, metric)

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

In [ ]:
neel_plotly.imshow(every_head_all_pos_act_patch_result, facet_col=0, facet_labels=["Output", "Query", "Key", "Value", "Pattern"], title="Activation Patching Per Head (All Pos)", xaxis="Head", yaxis="Layer", zmax=1, zmin=-1)

Weird. Good thing is I isolated the later heads which decide WHAT to move -- they are clearly v dependent on the OV circuit. This includes L8H11, L10H10, L9H9 and maybe L8H3.

L11H10 is a negative one and it only is correlated with the query.

But somehow nothing determining WHAT to move. Either it's because what to move is _quite obvious_ and there was only one choice, or more likely because I fucked up in designing the corrupted tokens. I think when designing the corrupted tokens I chose to replace `Carry` with `Friends` -- but the decision of WHERE to move information from does NOT change at all; what DOES change is only WHAT information to get from there (i.e. do we want letter F or C).

So maybe maybe I should change the corrupted tokens.

### Activation Patching 2

In [ ]:
# First define all the clean and corrupted tokens

clean_tokens_2 = model.to_tokens(prompts)

# Corrupted tokens are replacing the token 18 "Carry" by token 7 "Friends"
corrupted_tokens_2 = clean_tokens_2.clone()
corrupted_tokens_2[:, 7:10] = clean_tokens_2[:, 18:21]
corrupted_tokens_2[:, 18:21] = clean_tokens_2[:, 7:10]
for i in range(corrupted_tokens_2.shape[0]):
  s = prompts[i].split()[-4:-1]
  print(s)
  corrupted_tokens_2[i, 11] = model.to_single_token(s[0][0])
  corrupted_tokens_2[i, 12] = model.to_single_token(s[1][0]+s[2][0])

['Carry', 'On', 'Girls']
['Area', 'Effect', 'University']
['Bar', 'Beer', 'Parish']
['National', 'Executive', 'Board']
['Direct', 'Access', 'Corp']
['Electronic', 'Killer', 'Organisation']
['Zero', 'Waste', 'Initiative']
['Genetic', 'Mod', 'Plant']
['Political', 'Onion', 'Party']
['South', 'Oxford', 'School']
['Yellow', 'Fall', 'Board']
['Friends', 'Union', 'Party']
['Horror', 'Film', 'Society']
['X', 'Ray', 'Institute']
['World', 'Trade', 'Centre']
['International', 'Wii', 'Board']
['Reality', 'Doom', 'Arena']
['Killer', 'Game', 'Society']
['Question', 'Answer', 'Team']
['Motor', 'Assistance', 'Group']
['Technology', 'Application', 'Course']
['Local', 'Football', 'Club']
['Video', 'Games', 'Club']
['Object', 'Information', 'Network']
['Junior', 'Common', 'Room']
['Uniform', 'Distribution', 'Fans']


In [ ]:
model.to_single_token("F")

37

In [ ]:
s = prompts[0].split()
s[-4:-1]

['Carry', 'On', 'Girls']

In [ ]:
print("Clean string 0", model.to_string(clean_tokens_2[0]))
print("Corrupted string 0", model.to_string(corrupted_tokens_2[0]))

Clean string 0 <|endoftext|>Mary is a member of the Friends Union Party (FUP), as well as the Carry On Girls (
Corrupted string 0 <|endoftext|>Mary is a member of the Carry On Girls (COG), as well as the Friends Union Party (


In [ ]:
clean_logits_2, clean_cache_2 = model.run_with_cache(clean_tokens_2)
corrupted_logits_2, corrupted_cache_2 = model.run_with_cache(corrupted_tokens_2)

In [ ]:
clean_difference_2 = logits_to_ave_logit_diff(clean_logits_2, answer_tokens).cpu()
corrupted_difference_2 = logits_to_ave_logit_diff(corrupted_logits_2, answer_tokens).cpu()

In [ ]:
print(clean_difference_2, corrupted_difference_2)

tensor(5.1122) tensor(-4.5410)


In [ ]:
def metric_2(logits, answer_token_indices=answer_tokens):
  return (logits_to_ave_logit_diff(logits, answer_token_indices).cpu() - corrupted_difference_2) / (clean_difference_2 - corrupted_difference_2)

print(metric_2(clean_logits_2, answer_tokens))
print(metric_2(corrupted_logits_2, answer_tokens))

tensor(1.)
tensor(0.)


In [ ]:
every_block_result_2 = patching.get_act_patch_block_every(model, corrupted_tokens_2, clean_cache_2, metric_2)

  0%|          | 0/264 [00:00<?, ?it/s]

  0%|          | 0/264 [00:00<?, ?it/s]

  0%|          | 0/264 [00:00<?, ?it/s]

In [ ]:
neel_plotly.imshow(every_block_result_2, facet_col=0, facet_labels=["Residual Stream", "Attn Output", "MLP Output"], title="Activation Patching Per Block", xaxis="Position", yaxis="Layer", zmax=1, zmin=-1, x= [f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens_2[0]))])

In [ ]:
every_block_result_2[0].shape

torch.Size([12, 22])

Interesting. Now we're getting a bit more information.

Overall when patching the residual stream, we firstly observe as expected that patching the `Carry` token would immediately change the performance, and that in particular around layer 8-10 there appears to be a time where the information is moved towards the end token.

However, there is more behaviour: when the residual stream is patched on the first character of the example acronym, `C` becomes `F`, which inhibits the copying behaviour and therefore causes a loss in performance.

What *is* interesting is how it evolves for the Friends token -- replacing `Carry` in the corrupted tokens by `Friends` initially improves the performance, and then later worsens the performance. In other words, if in the prompt
`... Carry ... (C...), ... Friends ...(`
we replace `Carry` by `Friends` early, then the model actually predicts `C` as higher than before, whereas if we do so later it predicts `C` as lower than before. Something must have happened.

Hypothesis: the default behaviour is induction-ny: see a `(`, follow by `C` which is the one that is given, and somehow it says 'don't look at that token, just do induction'. If we patch it later it contains information that we need to do this capital-letter inference task, and replacing it with `Friends` somehow makes it inconsistent with the task?

Now on the attn output, clearly layers 8-10 are the important ones. Layer 11 somehow fucks the performance.

For the MLP output: it actually seems to be quite imp esp in the later layers of token 21! Though on token 18 it also seems important. Also MLP0 is important as always.

In [ ]:
every_head_all_pos_act_patch_result_2 = patching.get_act_patch_attn_head_all_pos_every(model, corrupted_tokens_2, clean_cache_2, metric_2)

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

In [ ]:
every_head_all_pos_act_patch_result_2.shape

torch.Size([5, 12, 12])

In [ ]:
neel_plotly.imshow(every_head_all_pos_act_patch_result_2, facet_col=0, facet_labels=["Output", "Query", "Key", "Value", "Pattern"], title="Activation Patching Per Head (All Pos)", xaxis="Head", yaxis="Layer", zmax=1, zmin=-1)

In [ ]:
head_labels = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
scatter(
    x=utils.to_numpy(every_head_all_pos_act_patch_result_2[3].flatten()),
    y=utils.to_numpy(every_head_all_pos_act_patch_result_2[0].flatten()),
    hover_name=head_labels,
    xaxis="Value Patch",
    yaxis="Output Patch",
    title="Scatter plot of output patching vs value patching",
)

In [ ]:
head_labels = [
    f"L{l}H{h}" for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)
]
scatter(
    x=utils.to_numpy(every_head_all_pos_act_patch_result_2[4].flatten()),
    y=utils.to_numpy(every_head_all_pos_act_patch_result_2[0].flatten()),
    hover_name=head_labels,
    xaxis="Query Patch",
    yaxis="Output Patch",
    title="Scatter plot of output patching vs query patching",
)

In more detail for the heads:

Output vs Value
- L8H3, L8H11, L10H10, L9H9 are the three OV-circuit heads, which clearly perform the task of moving relevant information.
- Also OV heavy L4H7 -- unclear what it does yet.
- L10H7 and L11H4 somehow inhibits. Maybe they promote an alternative behaviour? Should test different between just the first letter vs first + second letter.

Output vs Query
- L5H5, L5H8 etc. react strongly on a query patch, but not strongly on the output patch. Want to study what happened there.

In [ ]:
top_k = 15
top_heads_by_output_patch = torch.topk(
    every_head_all_pos_act_patch_result_2[0].abs().flatten(), k=top_k
).indices
# first_mid_layer = 6
first_late_layer = 8
early_heads = top_heads_by_output_patch[
    top_heads_by_output_patch < model.cfg.n_heads * first_late_layer # first_mid_layer
]
# mid_heads = top_heads_by_output_patch[
#     torch.logical_and(
#         model.cfg.n_heads * first_mid_layer <= top_heads_by_output_patch,
#         top_heads_by_output_patch < model.cfg.n_heads * first_late_layer,
#     )
# ]
late_heads = top_heads_by_output_patch[
    model.cfg.n_heads * first_late_layer <= top_heads_by_output_patch
]

early = visualize_attention_patterns(
    early_heads, cache, tokens[0], title=f"Top Early Heads"
)
# mid = visualize_attention_patterns(
#     mid_heads, cache, tokens[0], title=f"Top Middle Heads"
# )
late = visualize_attention_patterns(
    late_heads, cache, tokens[0], title=f"Top Late Heads"
)

HTML(early + late)

In [ ]:
HTML(visualize_attention_patterns(
    [5 * 12 + 5, 5*12 + 8], cache, tokens[0], title=f"Early weird heads"
))

**Conclusion of this section**

The comparison we wanted to draw was between the default induction behaviour we were hypothesising, whereby we get sth like the bigram `( -> C` if previously we had `(C`, and the behaviour of actually predicting the capital letter of the corresponding word.

At a high level, _almost_ all the action is on the `Carry` token and the last token, but there also is action on the `Friends` and `F` tokens.

Attention layers only seem to matter on layers 8-10 on the final token, whereas MLP layers seem to matter throughout the process and more so around L8-10 on the last token too.

On a per-head level, L8H11, L9H9, L10H10 seem to be the most important heads, which move information around (OV-circuit mainly). L8H11 seems to move mostly the FIRST token to the lpar, and then the second token is moved to the first token of the acronym, etc. L10H10 emphasises on the first one too but less so. L9H9 seems to put much bigger emphasis on the second and third token. **We hypothesise that it actually is the head that will allow the first token of the acronym to be 2 or 3 letters instead of 1.** (TODO: test this out).

L8H3 seems to move info from the first lparen `(` to `UP`: so clearly used to predict the next token in the acronym given the first one. **THIS IS A HYP**.

Moreover, we have some negative heads like L10H7: this one has an induction-ny behaviour of shoving Union Party (last two tokens of the first acronym) to `On Girls`.

Additional things to test:
- What if we compare 1-token vs 2-token predictions? Hyp is that L9H9 will be a negative factor, and even maybe L8H3.
- What do the MLPs do? For example does the L10 MLP inhibit L10H7's behaviour? How do the L8-L10 MLPs map from a token to its first character?
- Still unclear to me what's contributing the most to the behaviour on `F` and `Friends`. Let's come back to this.

In [ ]:
ALL_HEAD_LABELS = [f"L{i}H{j}" for i in range(model.cfg.n_layers) for j in range(model.cfg.n_heads)]
# every_head_act_patch_result_2 = patching.get_act_patch_attn_head_by_pos_every(model, corrupted_tokens_2, clean_cache_2, metric_2)
# every_head_act_patch_result_2 = einops.rearrange(every_head_act_patch_result_2, "act_type layer pos head -> act_type (layer head) pos")
# imshow(every_head_act_patch_result_2, facet_col=0, facet_labels=["Output", "Query", "Key", "Value", "Pattern"], title="Activation Patching Per Head (By Pos)", xaxis="Position", yaxis="Layer & Head", zmax=1, zmin=-1, x= [f"{tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens_2[0]))], y=ALL_HEAD_LABELS)

In [ ]:
attn_head_out_act_patch_results_2 = patching.get_act_patch_attn_head_out_by_pos(model, corrupted_tokens_2, clean_cache_2, metric_2)
attn_head_out_act_patch_results_2 = einops.rearrange(attn_head_out_act_patch_results_2, "layer pos head -> (layer head) pos")

  0%|          | 0/3168 [00:00<?, ?it/s]

In [ ]:
neel_plotly.imshow(attn_head_out_act_patch_results_2,
    yaxis="Head Label",
    xaxis="Pos",
    x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens_2[0]))],
    y=ALL_HEAD_LABELS,
    title="attn_head_out Activation Patching By Pos")

In [ ]:
pos_labels = [f"Layer {j}, {tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens_2[0])) for j in range(12)]
scatter(
    x=utils.to_numpy(every_block_result_2[1].flatten()),
    y=utils.to_numpy(every_block_result_2[0].flatten()),
    hover_name=pos_labels,
    xaxis="Attn Patch",
    yaxis="Res Patch",
    title="Scatter plot of Res patching vs Attn patching",
)

In [ ]:
pos_labels = [f"Layer {j}, {tok}_{i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens_2[0])) for j in range(12)]
scatter(
    x=utils.to_numpy(every_block_result_2[2].flatten()),
    y=utils.to_numpy(every_block_result_2[0].flatten()),
    hover_name=pos_labels,
    xaxis="MLP Patch",
    yaxis="Res Patch",
    title="Scatter plot of Res patching vs MLP patching",
)

So the behaviour of the MLP is sth I can explain as follows:
- Replacing "C" with "F" in the corrupted tokens will decrease the logit difference. This is because the inductive behaviour will increase the chance of getting "F".
- Replacing "Carry" with "Friends" eventually also decreases the logit difference for the same reason, or at least this is my guess. However in the beginning it actually increases the logit difference: this is probably because something inhibits the "Friends" token to be attended to the last _lparen_ or sth alike because it's unclear which task is required? (Q mark Q mark).

## A few initial experiments for induction vs 'formerly' vs 2-letter v 1-letter

In [27]:
# Test if I can get the first two characters.
strings = [p.split()[-4:-1] for p in prompts]
first_two = [s[0][0] + s[1][0] for s in strings]
indices = []
for i, s in enumerate(first_two):
  print(s)
  try:
    print(model.to_single_token(s))
    indices.append(i)
  except:
    print("no_token")
indices = torch.tensor(indices)
first_two_filtered = [first_two[i] for i in indices]

CO
8220
AE
14242
BB
15199
NE
12161
DA
5631
EK
no_token
ZW
no_token
GM
15548
PO
16402
SO
15821
YF
no_token
FU
38989
HF
29567
XR
no_token
WT
39386
IW
no_token
RD
35257
KG
no_token
QA
no_token
MA
5673
TA
5603
LF
no_token
VG
43490
OI
no_token
JC
34382
UD
8322


In [ ]:
print(len(indices))

17


In [40]:
def logit_lens_for_each_layer_1(indices=indices):
  acc_res, labels = cache.accumulated_resid(
      layer=-1, incl_mid=True, pos_slice=-1, return_labels=True, apply_ln=True
  )
  print(acc_res.shape) # [25, 26, 768]
  # scaled_residual_stack = cache.apply_ln_to_stack(
  #     acc_res, layer=-1, pos_slice=-1
  # )
  # print(scaled_residual_stack.shape) # layer, batch, d_model
  W_U = model.W_U
  layers_unembedded = einsum(
    "layer batch d_model, d_model d_vocab -> layer batch d_vocab",
    acc_res,
    W_U,
  )
  answer_tokens_cpu = answer_tokens.cpu()[indices]
  # print(answer_tokens_cpu[4, 0])
  # print(range(layers_unembedded_cpu.shape[1]))
  # layers_unembedded=F.softmax(layers_unembedded, dim=-1)
  # layers_unembedded = layers_unembedded.sum(dim=1)/layers_unembedded.shape[1]
  layers_unembedded_cpu = layers_unembedded.cpu()[:, indices]
  print(layers_unembedded_cpu.shape)
  correct_token_logit = torch.stack([layers_unembedded_cpu[:, i, answer_tokens_cpu[i, 0]] for i in range(layers_unembedded_cpu.shape[1])])
  incorrect_token_logit_1 = torch.stack([layers_unembedded_cpu[:, i, answer_tokens_cpu[i, 1]] for i in range(layers_unembedded_cpu.shape[1])])
  incorrect_token_logit_2 = torch.stack([layers_unembedded_cpu[:, i, model.to_single_token("formerly")] for i in range(layers_unembedded_cpu.shape[1])])
  incorrect_token_logit_3 = torch.stack([layers_unembedded_cpu[:, i, model.to_single_token(first_two_filtered[i])] for i in range(layers_unembedded_cpu.shape[1])])
  return correct_token_logit, incorrect_token_logit_1, incorrect_token_logit_2, incorrect_token_logit_3, labels
  # incorrect_token_logit = torch.tensor([layers_unembedded_cpu[:, i, answer_tokens_cpu[i, 1]] for i in layers_unembedded_cpu.shape[1]])
  # print(layers_unembedded[:, i, answer_tokens[i, 0]])
  # print(layers_unembedded[:, answer_tokens[:, 1], :].shape)

In [41]:
correct_logit, incorrect_logit_1, incorrect_logit_2, incorrect_logit_3, labels = logit_lens_for_each_layer_1()

torch.Size([25, 26, 768])
torch.Size([25, 17, 50257])


In [42]:
neel_plotly.line([correct_logit[12], incorrect_logit_1[12], incorrect_logit_2[12], incorrect_logit_3[12]], xaxis="Layer", yaxis="Logits", title="Correct Logit", x=labels)

In [43]:
neel_plotly.imshow(correct_logit-incorrect_logit_3)

In [44]:
top_k_per_layer = logit_lens_for_each_layer()

In [45]:
pd.DataFrame(top_k_per_layer)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,"(0_pre, [ mathemat, corrid, challeng])","(0_mid, [ART, 19, talk])","(1_pre, [see, aka, sic])","(1_mid, [see, aka, ...)])","(2_pre, [...), see, sic])","(2_mid, [...), aka, ?)])","(3_pre, [sic, ...), ?)])","(3_mid, [sic, aka, …)])","(4_pre, [sic, aka, ...)])","(4_mid, [sic, aka, formerly])",...,"(7_mid, [sic, aka, formerly])","(8_pre, [sic, aka, formerly])","(8_mid, [sic, aka, CW])","(9_pre, [CN, CW, C])","(9_mid, [CW, sic, CN])","(10_pre, [CW, CBC, CB])","(10_mid, [CB, CW, CC])","(11_pre, [CC, CB, CAR])","(11_mid, [CC, CF, CW])","(final_post, [CF, CC, CB])"
1,"(0_pre, [ mathemat, corrid, challeng])","(0_mid, [ART, change, sw])","(1_pre, [see, ...), sic])","(1_mid, [see, formerly, ...)])","(2_pre, [...), see, formerly])","(2_mid, [...), see, …)])","(3_pre, [...), see, sic])","(3_mid, [see, formerly, ...)])","(4_pre, [formerly, see, sic])","(4_mid, [formerly, sic, including])",...,"(7_mid, [formerly, sic, aka])","(8_pre, [formerly, sic, aka])","(8_mid, [formerly, sic, ARC])","(9_pre, [AS, formerly, ARC])","(9_mid, [AMP, AS, APS])","(10_pre, [APS, AS, AZ])","(10_mid, [AS, APS, AZ])","(11_pre, [AS, AZ, APS])","(11_mid, [AS, APS, AA])","(final_post, [AA, AH, AS])"
2,"(0_pre, [ mathemat, corrid, challeng])","(0_mid, [ART, Hide, API])","(1_pre, [see, ...), sic])","(1_mid, [...), see, sic])","(2_pre, [...), sic, see])","(2_mid, [...), sic, …)])","(3_pre, [sic, ...), ?)])","(3_mid, [sic, formerly, ...)])","(4_pre, [sic, formerly, which])","(4_mid, [sic, formerly, which])",...,"(7_mid, [sic, formerly, aka])","(8_pre, [sic, formerly, which])","(8_mid, [sic, formerly, BB])","(9_pre, [formerly, sic, BB])","(9_mid, [formerly, BB, BC])","(10_pre, [BB, BC, BT])","(10_mid, [BC, BB, Bar])","(11_pre, [BB, B, Bar])","(11_mid, [BB, B, BA])","(final_post, [BB, Bar, formerly])"
3,"(0_pre, [ mathemat, corrid, challeng])","(0_mid, [ART, sw, change])","(1_pre, [see, ...), sic])","(1_mid, [see, formerly, ...)])","(2_pre, [...), see, sic])","(2_mid, [...), see, …)])","(3_pre, [sic, see, ...)])","(3_mid, [see, formerly, sic])","(4_pre, [see, sic, formerly])","(4_mid, [see, including, formerly])",...,"(7_mid, [including, formerly, aka])","(8_pre, [formerly, which, sic])","(8_mid, [NC, formerly, NAS])","(9_pre, [NC, formerly, NAS])","(9_mid, [PU, NC, formerly])","(10_pre, [NK, N, NC])","(10_mid, [N, NK, NC])","(11_pre, [N, NE, NK])","(11_mid, [N, NE, NK])","(final_post, [N, NE, NC])"
4,"(0_pre, [ mathemat, corrid, challeng])","(0_mid, [ART, sw, Hide])","(1_pre, [see, ...), formerly])","(1_mid, [see, formerly, sic])","(2_pre, [...), see, sic])","(2_mid, [sic, ...), formerly])","(3_pre, [sic, ...), formerly])","(3_mid, [sic, formerly, aka])","(4_pre, [sic, formerly, which])","(4_mid, [formerly, sic, which])",...,"(7_mid, [formerly, aka, sic])","(8_pre, [formerly, aka, sic])","(8_mid, [DD, DH, D])","(9_pre, [DH, DD, DM])","(9_mid, [DL, AK, DOC])","(10_pre, [DL, DR, DP])","(10_mid, [DT, DL, DC])","(11_pre, [DA, DC, DL])","(11_mid, [DA, DC, DD])","(final_post, [DA, DC, DT])"
5,"(0_pre, [ mathemat, corrid, challeng])","(0_mid, [ART, sw, CPU])","(1_pre, [see, ...), sic])","(1_mid, [see, ...), ?)])","(2_pre, [...), see, ?)])","(2_mid, [...), …), see])","(3_pre, [sic, ...), see])","(3_mid, [sic, formerly, …)])","(4_pre, [sic, formerly, including])","(4_mid, [sic, formerly, aka])",...,"(7_mid, [formerly, aka, sic])","(8_pre, [formerly, aka, sic])","(8_mid, [EH, formerly, E])","(9_pre, [EH, EC, EM])","(9_mid, [EH, EM, EB])","(10_pre, [EH, EB, EMA])","(10_mid, [EH, EB, EF])","(11_pre, [EH, EFF, EA])","(11_mid, [EFF, EH, EA])","(final_post, [EFF, EM, EMA])"
6,"(0_pre, [ mathemat, corrid, challeng])","(0_mid, [ART, 19, Hide])","(1_pre, [see, ...), sic])","(1_mid, [formerly, see, ...)])","(2_pre, [...), see, formerly])","(2_mid, [formerly, ...), aka])","(3_pre, [sic, ...), ?)])","(3_mid, [formerly, sic, aka])","(4_pre, [sic, formerly, which])","(4_mid, [formerly, sic, which])",...,"(7_mid, [sic, formerly, aka])","(8

In [46]:
per_layer_residual, labels = cache.decompose_resid(
    layer=-1, pos_slice=-1, return_labels=True
)
print(per_layer_residual.shape, labels)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, cache)
line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

torch.Size([26, 26, 768]) ['embed', 'pos_embed', '0_attn_out', '0_mlp_out', '1_attn_out', '1_mlp_out', '2_attn_out', '2_mlp_out', '3_attn_out', '3_mlp_out', '4_attn_out', '4_mlp_out', '5_attn_out', '5_mlp_out', '6_attn_out', '6_mlp_out', '7_attn_out', '7_mlp_out', '8_attn_out', '8_mlp_out', '9_attn_out', '9_mlp_out', '10_attn_out', '10_mlp_out', '11_attn_out', '11_mlp_out']


## Studying MLPs

In [47]:
directions = model.tokens_to_residual_directions(answer_tokens)

In [48]:
print(directions.shape)

torch.Size([26, 2, 768])


In [63]:
directions[:, 0, :].shape

torch.Size([26, 768])

In [121]:
acc_res, labels = cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True, apply_ln=True
)

In [122]:
labels[-5:-2]

['10_pre', '10_mid', '11_pre']

In [123]:
acc_res[-5:-2].shape

torch.Size([3, 26, 768])

In [124]:
correct_dot_product = einsum("layer prompt d_model, prompt d_model -> prompt layer", acc_res[-5:-2], directions[:, 0, :])
error_dot_product = einsum("layer prompt d_model, prompt d_model -> prompt layer", acc_res[-5:-2], directions[:, 0, :]-directions[:, 1, :])

In [125]:
neel_plotly.line(correct_dot_product, x=labels[-5:-2], xaxis="Hookpoint", yaxis="Logit for the correct prediction")

In [126]:
neel_plotly.line(error_dot_product, x=labels[-5:-2], xaxis="Hookpoint", yaxis="Difference")

Now let's ablate L10H7

In [83]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [139]:
def hook_function(
    head_vector: Float[torch.Tensor, "batch pos head_index d_head"],
    hook,
    head_index,
    clean_cache=cache,
):
    head_vector[:, :, head_index, :] = 0
    return head_vector

with model.hooks(fwd_hooks=[(utils.get_act_name("z", 10, "attn"), partial(hook_function, head_index=10))]):

  ablated_10_10_run, ablated_cache = model.run_with_cache(
    tokens,
    return_type="logits",
  )

In [140]:
ablated_10_10_run.shape

torch.Size([26, 22, 50257])

In [141]:
logit_tops, top_idx = torch.topk(ablated_10_10_run[:, -1, :], 3, dim=-1)
print(top_idx.shape)
results = [model.to_str_tokens(top_idx[prompt]) for prompt in range(len(answers))]

torch.Size([26, 3])


In [142]:
print(results)

[['C', 'CC', 'CL'], ['A', 'AS', 'AA'], ['B', 'BB', 'Bar'], ['N', 'NE', 'NC'], ['D', 'DA', 'A'], ['E', 'EM', 'ED'], ['Z', 'W', 'N'], ['G', 'GM', 'GN'], ['P', 'PN', 'PC'], ['S', 'LS', 'SO'], ['Y', 'W', 'G'], ['F', 'FS', 'FT'], ['H', 'F', 'FI'], ['X', 'H', 'Y'], ['W', 'WW', 'WC'], ['I', 'II', 'IB'], ['R', 'RP', 'VR'], ['K', 'R', 'G'], ['Q', 'R', 'RS'], ['M', 'MC', 'MAP'], ['T', 'TS', 'S'], ['L', 'LS', 'LE'], ['V', 'VA', 'W'], ['O', 'OR', 'OV'], ['V', 'J', 'K'], ['U', 'V', 'UT']]


In [143]:
acc_res_abl_10_10, labels = ablated_cache.accumulated_resid(
    layer=-1, incl_mid=True, pos_slice=-1, return_labels=True, apply_ln=True
)

In [144]:
correct_dot_product_abl = einsum("layer prompt d_model, prompt d_model -> prompt layer", acc_res_abl_10_10[-5:-2], directions[:, 0, :])
error_dot_product_abl = einsum("layer prompt d_model, prompt d_model -> prompt layer", acc_res_abl_10_10[-5:-2], directions[:, 0, :]-directions[:, 1, :])

In [145]:
neel_plotly.line(correct_dot_product_abl, x=labels[-5:-2], xaxis="Hookpoint", yaxis="Logit for the correct prediction")

In [146]:
neel_plotly.line(error_dot_product_abl, x=labels[-5:-2], xaxis="Hookpoint", yaxis="Difference")